# Week 1

## Exercise 1 — Less than

The body of the function can be done in a single line with a list comprehension

In [1]:
def less_than(original, n):
    return [e for e in original if e < n]

print(less_than([4, 1, 3, 2, 5, 0, -2], 2))
print(less_than([], 10))
print(less_than(list(range(20)), 5))

[1, 0, -2]
[]
[0, 1, 2, 3, 4]


## Exercise 2 — A deck of cards
We first create the deck

In [2]:
deck = []
for value in range(1, 14):
    for suit in ('H', 'S', 'D', 'C'):
        deck.append((value, suit))

# Or by list comprehension
deck = [(v, s) for v in range(1, 14) for s in ('H', 'S', 'D', 'C')]

Next we shuffle it and use `.pop` to draw cards. I prefer `pop` over slicing here, because then the cards are actually removed from the deck. A slice `hand = deck[:13]` would draw 13 cards, but not actually remove them from the deck, meaning we would now have two copies of the draw cards in existence.

In [3]:
import random
random.shuffle(deck)
hand = [deck.pop() for _ in range(13)]

Next we want to sort the cards we drew, once by values and once by suits. We want to perform the value sort *first*, because then we can sort by suits and preserve the value sorting.

To sort by the first element of a tuple we simply use the `list.sort()` method. To sort by the second element of the tuple, we need to use the `key` kwarg of the `.sort`-metod. Where you can either use `operator.itemgetter`:
```Python
hand.sort(key=itemgetter(1))
```
or a simple lambda function:

In [4]:
hand.sort()
hand.sort(key=lambda e: e[1])

Then we print to verify:

In [5]:
for card in hand:
    print(card)

(1, 'C')
(8, 'C')
(12, 'C')
(13, 'C')
(12, 'D')
(13, 'D')
(7, 'H')
(9, 'H')
(13, 'H')
(5, 'S')
(6, 'S')
(12, 'S')
(13, 'S')


## Exercise 3 — Letter Counts

This exercise can be solved in many ways. First we show perhaps the most intuitive solution, we loop over all the characters, and add them to the dictionary. We use an if-test to add the first occurence of each character, and incrementing for later occurences. Note also that we loop over `text.lower()`, so we only get lowercase letters

In [6]:
def count_chars(text):
    count = {}
    for char in text.lower():
        if char not in count:
            count[char] = 1
        else:
            count[char] += 1
    return count

We can avoid the if-test in two ways. We can either use a `collections.defaultdict`

In [7]:
from collections import defaultdict

def count_chars_defaultdict(text):
    count = defaultdict(int)
    for char in text.lower():
        count[char] += 1
    return count

As the name implies, the defaultdict is a dictionary with a "default" value, so we can pretend `count[char]` exists, even from the first occurence. If it doesn't exist, it just defaults to 0.

The other way to avoid the if-test is to use the `.get()` method, which can also be given a default option. `count.get(char, 0)` will return the value of the element if it is present in the dictionary, and 0 if it is not. This can be used as follows:

In [8]:
def count_chars_get(text):
    count = {}
    for char in text.lower():
        count[char] = count.get(char, 0) + 1
    return count

Finally, there are some built-in functionality that does this. We have the option of using the string `.count` method, or the `collections.Counter` object:

In [9]:
example = "Hello, World!"

assert(count_chars(example) == count_chars_defaultdict(example))

In [10]:
from collections import Counter

def count_chars_str_count(text):
    text = text.lower()
    return {char: text.count(char) for char in set(text)}

def count_chars_counter(text):
    return Counter(text.lower())

Finally, to print in sorted order, we simply use `sorted` on the dictionary items when looping:

In [11]:
example = "Hello, world!"
for char, count in count_chars_counter(example).items():
    print(f'{char:3}{count:10}')

h           1
e           1
l           3
o           2
,           1
            1
w           1
r           1
d           1
!           1


In [13]:
example = "Hello, World!"
for char, count in sorted(count_chars(example).items()):
    print(f"{char:3}{count:10}")

            1
!           1
,           1
d           1
e           1
h           1
l           3
o           2
r           1
w           1


While to sort by number of occurences we have to specify that it is the second element we sort by. This can either be done by `operator.itemgetter`:
```Python
sorted(count.items(), key=itemgetter(1), reverse=True)
```
or using a simple lambda function
```Python
sorted(count.items(), key=lambda elem: elem[1], reverse=True)
```
In either case, we need to also use the `reverse` keyword, as sorting normally sorts in increasing order.

## Exercise 4 — Factorizing a number

First we handle the special case of 1, as this is a bit tricky. Next we need to loop from 2 to $n$ to look for divisors. Every time we find a divisor we divide it out and add it to our list of divisors. 

In [13]:
def factorize(n):
    if n == 1:
        return [1]
    
    divisors = []
    while n != 1:
        for d in range(2, n+1):
            if n % d == 0:
                divisors.append(d)
                n = n//d
                break #added this H19. More correct?
    return divisors

print(factorize(18))
print(factorize(23))
print(factorize(412415))

[2, 3, 3]
[23]
[5, 82483]


## Exercise 5 — Name Scores

Here we first need to read all the names from the file. The best way to do this is to use a *context guard*, meaning we use the `with` keyword. Then we do not need to remember to close the file manually. How best to read the text and convert it to a list of names depends on the original input. In this case we first remove all `"` characters, and then split in on the commas.

In [14]:
with open('names.txt', 'r') as infile:
        text = infile.read()
        text = text.replace('"', '')
        names = text.split(',')

# Check if reading went correctly
print(names[:5])

['MARY', 'PATRICIA', 'LINDA', 'BARBARA', 'ELIZABETH']


Next we need to sort the names in alphabetical order, this can be done using the `.sort()` list method

In [15]:
names.sort()
print(names[:5])

['AARON', 'ABBEY', 'ABBIE', 'ABBY', 'ABDUL']


Now we need to find the "alphabetical value" of each name. To do this we create a function that converts a letter into its location in the alphabet. We can import the alphabet as a string from the `string` package. To find the position of a character in a string (or an element in a list), we can use the `.find` method, which returns the index of the first occurence. Since Python starts counting at 0, but we want A to correspond to 1, we add 1 to the result.

In [16]:
import string

ALPHABET = string.ascii_uppercase

def letter_position(letter):
    return ALPHABET.find(letter) + 1

Next we define the function that converts each letter in a name independently, then sum the resulting integers. We test the function with the example given, namely that "COLIN" should have a value of 53.

In [17]:
def alphabetical_value(name):
    return sum([letter_position(letter) for letter in name])

print(alphabetical_value("COLIN"))

53


Now we want to go through the whole list of names, find the value of each one and multiply it by its position in the list of names

In [18]:
total = 0
for i, name in enumerate(names):
    total += alphabetical_value(name) * (i+1)
    
print(total)

871198282


Which is our final answer. Writing the whole thing out as a single program can be done as follows:

In [19]:
import string


def letter_position(letter):
    return string.ascii_uppercase.find(letter) + 1


def alphabetical_value(name):
    return sum([letter_position(letter) for letter in name])


def total_value(names):
    total = 0
    for i, name in enumerate(names):
        total += alphabetical_value(name) * (i+1)
    return total


def read_names(filepath):
    with open(filepath, 'r') as infile:
        text = infile.read()
        text = text.replace('"', '')
        names = text.split(',')
    return names


if __name__ == "__main__":
    names = read_names("names.txt")
    names.sort()
    print(total_value(names))

871198282
